In [1]:
import fastai
fastai.__version__

'1.0.54'

In [ ]:
!sh download_fastai_sentiment_model.sh

In [ ]:
!du -h sentiment_model.tgz

In [2]:
!tar xzvf sentiment_model.tgz

x storage/sentiment/
x storage/sentiment/models/
x storage/sentiment/models/third.pth
x storage/sentiment/data_clas.pkl


In [3]:
!mv storage model

In [4]:
from pathlib import Path
from fastai.text import load_data, text_classifier_learner, AWD_LSTM
bs=48
path=Path('./model/sentiment')
data_clas = load_data(path, 'data_clas.pkl', bs=bs)
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)


In [5]:
learn.load('third')
learn.model.eval()

SequentialRNN(
  (0): MultiBatchEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(30832, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(30832, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0): RNNDropout()
        (1): RNNDropout()
        (2): RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Dropout(p=0.2)
      (2): Linear(in_features=1200, out_features=50, bias=True)
      (3): ReLU(inplace)
      (4): BatchNorm1d(50, eps=1e-05, momentum=

In [6]:
example_message = "I don't know why I keep focusing on negative things. What a bummer!"

In [7]:
learn.predict(example_message)

(Category neg, tensor(0), tensor([0.9182, 0.0818]))

In [8]:
learn.predict('I love this workshop')

(Category pos, tensor(1), tensor([0.0385, 0.9615]))

In [9]:
from typing import Tuple
def predict_sentiment(sentence:str) -> Tuple[str, float]:
    categorical, class_id, scores = learn.predict(sentence)
    score = round(scores[class_id].item(), 4)
    return "negative" if class_id == 0 else "positive", score

In [10]:
predict_sentiment(example_message)

('negative', 0.9182)

# Add step to the pipeline

In [11]:
import redis
r = redis.Redis()
sub = r.pubsub(ignore_subscribe_messages=True)
sub.subscribe(['NER'])

In [12]:
import mq

In [18]:
import tqdm

In [21]:
for message in tqdm.tqdm(sub.listen()):
    data = mq.read_message_data(message)
    print(data)
    sentiment, score = predict_sentiment(data['sentence'])
    print(sentiment, score)
    for e in data['entities']:
        output = dict(
            entity=e,
            sentiment=sentiment,
            score=score
        )
        r.publish('ready', mq.serialize_message_data(output))



0it [00:00, ?it/s]
1it [00:03,  3.29s/it]

{'sentence': ' Why are you not convinced by John Travolta?', 'entities': ['John Travolta']}
negative 0.975


ConnectionError: Error while reading from socket: ('Connection closed by server.',)

### Fallback always-happy sentiment in case the WiFi fails

In [ ]:
import random

In [ ]:
def decide_sentiment(sentence:str) -> Tuple[str, float]:
    return 'positive', round(random.uniform(.5, 1), 3)